# Build a decision tree

## Several reminders of this code:

   1. This is a rewriten code references to tutorial from [Josh Gordon](https://github.com/random-forests), since my original code is not in perfect structure.
   2. This code includes ID3 and CART algorithm. 
   3. Input data should be pandas DataFrame
   3. Strongly suggest using hot encoding for categorical data, especially for those categorical features with integers.
   4. Feel free to contact me if you think there are some problems.
   5. ***Pruning is not included yet, I will work on it very soon.***
   6. ***I will add more hyperparameters into this code.***

In [1487]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import collections
%matplotlib inline

In [1488]:
def is_numeric(value):
    # This function is to determine if a value is numerical
    return isinstance(value, int) or isinstance(value, float)

def unique_vals(dataset, feature):
    """Find the unique values for a column in a dataset."""
    return sorted(set(dataset[feature]))

def class_counts(dataset):
        """
        Parameters
        ------------------
        dataset: {DataFrame}, shape = [m_samples, n_features]
            Dateset matrix, where 'm_samples' is the number of samples 
            and 'n_features' is the number of features

        Return
        ------------------
        classCounts: dictionary {class: counts}

        """
        classCounts = collections.defaultdict(int)

        # the number of unique elements and their occurrence
        for label in dataset.iloc[:,-1]:
            if label not in classCounts:
                classCounts[label] = 0
            classCounts[label] += 1
        return classCounts

In [1489]:
class Split:

    def __init__(self, dataset, feature, splitPoint):
        self.dataset = dataset
        self.feature = feature
        self.splitPoint = splitPoint

    def match(self, comparision_sample):
        # This is the function that compare the plit point we choosed to a given value
        # If it is numerical, then True if splitPoint >= comparision_sample, False if splitPoint < comparision_sample
        # If it is categorical, then True if splitPoint == comparision_sample, False if splitPoint != comparision_sample
        val = comparision_sample[self.feature]

        if is_numeric(val):
            return self.splitPoint <= val
        else:
            return self.splitPoint == val 
        
    def __repr__(self):
        # This is just a helper method to print
        # the question in a readable format.
        header = list(self.dataset.columns)
        condition = "=="
        if is_numeric(self.splitPoint):
            condition = ">="
        return "Is %s %s %s?" % (
            header[header==self.feature], condition, str(self.splitPoint))

class Leaf:
    """ A leaf node

    A leaf node holds data including unique values and their counts in a dictionary.
    """
    def __init__(self, dataset):
        self.prediction = class_counts(dataset)


class Decision_Node:
    """A decision node

    A decision node holds the split method and its two child tree.
    """
    def __init__(self, split, left_tree, right_tree):
        self.split = split
        self.left_tree = left_tree
        self.right_tree = right_tree

In [1490]:



class DecisionTreeClassifier:
    
    def __init__(self, criterion = 'entropy'):
        self.criterion = criterion
        
        if self.criterion == 'gini':
            self.info_gain = self._gini_gain
        elif self.criterion == 'entropy':
            self.info_gain = self._entropy_gain
    
    
    def partition(self, dataset, split):
        """partition the dataset into left and right

        For each value a in a feature, compare it to the split point. Partition the dataset into two subsets.

        """
        
        left_set = dataset[split.match(dataset)]
        
        right_set = dataset[split.match(dataset)==False]

        return left_set, right_set
    
    def gini_Impurity(self, dataset):
        """
        Parameters
        ------------------

        Return
        ------------------
        giniImpurity: float

        """
        classCounts = class_counts(dataset)
        # calculate gini index
        gini = 1.0
        for key in classCounts:
            # calculate occurrence
            prob = float(classCounts[key]/(len(dataset)*1.0))
            # calculate entropy
            gini -= prob ** 2 

        return gini
    
    def _gini_gain(self, dataset, left, right):
        """Gini gain

        """
        gini = self.gini_Impurity(dataset)
        p = float(len(left) / (len(left) +len(right)))
        
        return gini - float(p * self.gini_Impurity(left)) - float((1-p) * self.gini_Impurity(right)) 
    
    def calcShannonEntropy(self, dataset):
        """
        Parameters
        ------------------
        y_labels: {array-like}, shape = [n_samples, 1]
            Dateset matrix, where 'n_samples' is the number of samples 
            and one column of labels
        
        Return
        ------------------
        shannonEntropy: float
        
        """
        m, n = dataset.shape
        
        labelCounts = collections.defaultdict(int)
        
        classCounts = class_counts(dataset)
            
        # calculate shannon entropy
        shannonEntropy = 0.0
        for key in classCounts:
            # calculate occurrence
            prob = float(classCounts[key]/m)
            if prob == 0:
                continue
            # calculate entropy
            shannonEntropy -= prob * np.log2(prob)
        
        return shannonEntropy
    
    def _entropy_gain(self, dataset, left, right):
        """Entropy gain

        """
        entropy = self.calcShannonEntropy(dataset)
        p = float(len(left) / (len(left) +len(right)))
        
        return entropy - float(p*self.calcShannonEntropy(left)) -  float((1-p)*self.calcShannonEntropy(right))
   

    def find_best_split(self, dataset):
        """
        Parameters
        ------------------
        X_data: {array-like} discrete features
        preprocessed dataset, could not deal with continuous features, 
        and categorical feature should be better as binary form

        y_labels: {array-like}

        Return
        ------------------
        best_gain: the maximum gini index gain
        best_split: the feature and split point that get the maximum gini index gain

        """
        # Exclude Labels
        features = list(dataset.columns[:-1])


        best_gain = 0.0
        best_split = None

        # Loops:
        # First for loop: features
        # Second for loop: unique values in a feature.

        for feature in features:

            unique_val = unique_vals(dataset, feature)

            for val in unique_val:

                split = Split(dataset, feature, val)

                left_set, right_set = self.partition(dataset, split)

                # Skip this split if it doesn't divide the dataset
                if len(left_set) == 0 or len(right_set) == 0:
                    continue

                gain = self.info_gain(dataset, left_set, right_set)

                if gain >= best_gain:
                    best_gain = gain 
                    best_split = split

        return best_gain, best_split

    
    def fit(self,dataset):
        """

        """

        #Step1: Find the best split feature and point, and create a root.
        gain, split = self.find_best_split(dataset)

        # If there is no gain, or gain is less than a threshold
        # We will not split any more
        # And left it as a leaf
        if gain == 0:
            return Leaf(dataset)

        # Partition the dataset into two sub-trees
        left_set, right_set = self.partition(dataset, split)

        # Recursively build sub-trees, start from left to right
        left_tree = self.fit(left_set)
        right_tree = self.fit(right_set)
        #
        
        return Decision_Node(split, left_tree, right_tree)
    
    def print_tree(self, node, spacing=""):
        
        """Copy from
        World's most elegant tree printing function.
        """

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            print (spacing + "Predict", self.print_leaf(node.prediction))
            return

        # Print the question at this node
        print (spacing + str(node.split))

        # Call this function recursively on the true branch
        print (spacing + '--> True:')
        self.print_tree(node.left_tree, spacing + "  ")

        # Call this function recursively on the false branch
        print (spacing + '--> False:')
        self.print_tree(node.right_tree, spacing + "  ")

    def classify(self, dataset, node):
        
        """See the 'rules of recursion' above."""

        # Base case: we've reached a leaf
        if isinstance(node, Leaf):
            return node.prediction

        # Decide whether to follow the true-branch or the false-branch.
        # Compare the feature / value stored in the node,
        # to the example we're considering.
        if node.split.match(dataset):
            return self.classify(dataset, node.left_tree)
        else:
            return self.classify(dataset, node.right_tree)
        
    def print_leaf(self, counts):
        """A nicer way to print the predictions at a leaf."""
        total = sum(counts.values()) * 1.0
        probs = {}
        for lbl in counts.keys():
            probs[lbl] = str(int(counts[lbl] / total * 100)) + "%"
        return probs
    
    def save_tree(self, inputTree, filename):
        import pickle
        fw = open(filename, 'wb')
        pickle.dump(inputTree, fw)
        fw.close()
        
    def read_tree(self, filename):
        import pickle
        tr = open(filename,'rb')
        return pickle.load(tr)
    
    def predict(self, dataset, node):
        predictions = []
        m,n = dataset.shape
        for i in range(m):
            leaf = self.classify(dataset.iloc[i],node)
#             print('The prediction is: {}'.format(self.print_leaf(leaf)))
            predictions.append(self.print_leaf(leaf).keys())

        return pd.DataFrame(predictions)




In [1491]:
DATA = pd.read_csv('mushrooms.csv')
X = DATA.iloc[:,1:]
y = DATA.iloc[:,0]
# data['labels'] = labels

In [1492]:
X.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-surface-below-ring,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat
0,x,s,n,t,p,f,c,n,k,e,...,s,w,w,p,w,o,p,k,s,u
1,x,s,y,t,a,f,c,b,k,e,...,s,w,w,p,w,o,p,n,n,g
2,b,s,w,t,l,f,c,b,n,e,...,s,w,w,p,w,o,p,n,n,m
3,x,y,w,t,p,f,c,n,n,e,...,s,w,w,p,w,o,p,k,s,u
4,x,s,g,f,n,f,w,b,k,t,...,s,w,w,p,w,o,e,n,a,g


In [1493]:
from sklearn.model_selection import train_test_split

In [1494]:
X_train, X_test, y_train, y_test = train_test_split( X, y, test_size=0.33, random_state=42)

In [1495]:
data = pd.concat([X_train,y_train],axis=1)

In [1496]:
data.head()

,cap-shape,cap-surface,cap-color,bruises,odor,gill-attachment,gill-spacing,gill-size,gill-color,stalk-shape,...,stalk-color-above-ring,stalk-color-below-ring,veil-type,veil-color,ring-number,ring-type,spore-print-color,population,habitat,class
3244,f,f,e,t,n,f,c,b,n,t,...,p,g,p,w,o,p,k,y,d,e
3739,x,f,g,f,c,f,w,n,p,e,...,w,w,p,w,o,p,k,v,d,p
2154,x,f,e,t,n,f,c,b,n,t,...,w,g,p,w,o,p,n,y,d,e
1344,f,s,w,f,n,f,w,b,n,t,...,w,w,p,w,o,e,n,s,g,e
7852,k,y,n,f,s,f,c,n,b,t,...,w,w,p,w,o,e,w,v,l,p


In [1497]:
test = pd.concat([X_test,y_test],axis=1)

In [1499]:
dt = DecisionTreeClassifier()

In [1500]:
tree = dt.fit(data)

In [1501]:
dt.save_tree(tree,'tree')

In [1502]:
tree = dt.read_tree('tree')

In [1503]:
dt.print_tree(tree)

Is cap-shape == n?
--> True:
  Is cap-shape == r?
  --> True:
    Predict {'p': '100%'}
  --> False:
    Is cap-shape == y?
    --> True:
      Is cap-shape == y?
      --> True:
        Predict {'e': '100%'}
      --> False:
        Predict {'p': '100%'}
    --> False:
      Is cap-shape == n?
      --> True:
        Is cap-shape == c?
        --> True:
          Predict {'p': '100%'}
        --> False:
          Predict {'e': '100%'}
      --> False:
        Predict {'e': '100%'}
--> False:
  Is cap-shape == t?
  --> True:
    Is cap-shape == c?
    --> True:
      Predict {'e': '100%'}
    --> False:
      Is cap-shape == y?
      --> True:
        Predict {'e': '100%'}
      --> False:
        Is cap-shape == d?
        --> True:
          Predict {'e': '100%'}
        --> False:
          Predict {'p': '100%'}
  --> False:
    Predict {'p': '100%'}


In [1504]:
prediction = dt.predict(test,tree)

In [1505]:
prediction

,0
0,e
1,p
2,p
3,e
4,p
...,...
2676,p
2677,e
2678,e
2679,e


In [1506]:
data['class']

3244    e
3739    p
2154    e
1344    e
7852    p
       ..
5226    p
5390    e
860     e
7603    p
7270    e
Name: class, Length: 5443, dtype: object

In [1507]:
from sklearn.metrics import confusion_matrix

In [1509]:
confusion_matrix(test['class'],prediction)

array([[1378,    0],
       [   0, 1303]])

# Too good to be true, only because lack of pruning